In [219]:
import requests
import pandas as pd
from config import api_key


In [220]:
# Top 10 world cities
CITIES = [
    {"city": "Tokyo", "lat": 35.6895, "lon": 139.6917, "population": 13929286},
    {"city": "Delhi", "lat": 28.7041, "lon": 77.1025, "population": 27882721},
    {"city": "Shanghai", "lat": 31.2304, "lon": 121.4737, "population": 24256800},
    {"city": "São Paulo", "lat": -23.5505, "lon": -46.6333, "population": 21571281},
    {"city": "Mumbai", "lat": 19.0760, "lon": 72.8777, "population": 21558265},
    {"city": "Mexico City", "lat": 19.4326, "lon": -99.1332, "population": 21782378},
    {"city": "Beijing", "lat": 39.9042, "lon": 116.4074, "population": 21516000},
    {"city": "Osaka", "lat": 34.6937, "lon": 135.5023, "population": 19222665},
    {"city": "New York", "lat": 40.7128, "lon": -74.0060, "population": 18804000},
    {"city": "Cairo", "lat": 30.0444, "lon": 31.2357, "population": 20095952}
  ]

In [221]:

# Create an empty list to store air pollution data for each city
air_pollution_data_list = []

# Loop through the cities and make API calls for historical air pollution data
for city in CITIES:
    lat = city["lat"]
    lon = city["lon"]
    city_name = city["city"]

    # Set the start and end dates for historical data
    start = 1641016800  # Start date 01-01-2022
    end = 1672552799  #  End date 31-12-2022

    # Make API call
    url = f"http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start}&end={end}&appid={api_key}"
    response = requests.get(url)

    # Check if API call was successful
    if response.status_code == 200:
        # Extract air pollution data from response
        air_pollution_data = response.json()

        # Append the air pollution data for this city to the final list
        air_pollution_data_list.append(air_pollution_data)
    

In [ ]:
air_pollution_df = pd.json_normalize(air_pollution_data_list,record_path="list", meta=['coord'])
air_pollution_df

,dt,main.aqi,components.co,components.no,components.no2,components.o3,components.so2,components.pm2_5,components.pm10,components.nh3,coord
0,1641016800,2,270.37,1.98,12.68,87.26,12.76,0.62,1.38,0.57,"{'lon': 139.6917, 'lat': 35.6895}"
1,1641020400,1,280.38,1.31,20.05,78.68,16.93,0.78,1.79,0.77,"{'lon': 139.6917, 'lat': 35.6895}"
2,1641024000,1,297.07,0.12,27.42,69.38,20.03,0.94,2.09,1.01,"{'lon': 139.6917, 'lat': 35.6895}"
3,1641027600,1,310.42,0.01,30.85,62.94,21.93,1.06,2.24,1.19,"{'lon': 139.6917, 'lat': 35.6895}"
4,1641031200,1,320.43,0.02,36.67,55.79,26.23,1.17,2.48,1.38,"{'lon': 139.6917, 'lat': 35.6895}"
...,...,...,...,...,...,...,...,...,...,...,...
85915,1672534800,2,317.10,0.02,23.99,28.61,12.76,23.35,48.37,2.50,"{'lon': 31.2357, 'lat': 30.0444}"
85916,1672538400,2,313.76,0.02,22.28,27.18,12.28,23.23,43.94,2.47,"{'lon': 31.2357, 'lat': 30.0444}"
85917,1672542000,2,310.42,0.03,21.25,28.25,12.16,22.07,39.83,2.41,"{'lon': 31.2357, 'lat': 30.0444}"
85918,1672545600,2,313.76,0.04,22.62,28.25,13.83,20.83,36.78,2.53,"{'lon': 31.2357, 'lat': 30.0444}"


In [ ]:
air_pollution_df.columns

Index(['dt', 'main.aqi', 'components.co', 'components.no', 'components.no2',
       'components.o3', 'components.so2', 'components.pm2_5',
       'components.pm10', 'components.nh3', 'coord'],
      dtype='object')

In [ ]:
air_pollution_df

,dt,main.aqi,components.co,components.no,components.no2,components.o3,components.so2,components.pm2_5,components.pm10,components.nh3,coord
0,1641016800,2,270.37,1.98,12.68,87.26,12.76,0.62,1.38,0.57,"{'lon': 139.6917, 'lat': 35.6895}"
1,1641020400,1,280.38,1.31,20.05,78.68,16.93,0.78,1.79,0.77,"{'lon': 139.6917, 'lat': 35.6895}"
2,1641024000,1,297.07,0.12,27.42,69.38,20.03,0.94,2.09,1.01,"{'lon': 139.6917, 'lat': 35.6895}"
3,1641027600,1,310.42,0.01,30.85,62.94,21.93,1.06,2.24,1.19,"{'lon': 139.6917, 'lat': 35.6895}"
4,1641031200,1,320.43,0.02,36.67,55.79,26.23,1.17,2.48,1.38,"{'lon': 139.6917, 'lat': 35.6895}"
...,...,...,...,...,...,...,...,...,...,...,...
85915,1672534800,2,317.10,0.02,23.99,28.61,12.76,23.35,48.37,2.50,"{'lon': 31.2357, 'lat': 30.0444}"
85916,1672538400,2,313.76,0.02,22.28,27.18,12.28,23.23,43.94,2.47,"{'lon': 31.2357, 'lat': 30.0444}"
85917,1672542000,2,310.42,0.03,21.25,28.25,12.16,22.07,39.83,2.41,"{'lon': 31.2357, 'lat': 30.0444}"
85918,1672545600,2,313.76,0.04,22.62,28.25,13.83,20.83,36.78,2.53,"{'lon': 31.2357, 'lat': 30.0444}"


In [ ]:
# Define a function to split 'coord' column into 'longitude' and 'latitude'
def split_coord(coord):
    return pd.Series([coord['lon'],coord['lat']])

# Apply the function to 'coord' column to create 'longitude' and 'latitude' columns
air_pollution_df[['longitude', 'latitude']] = air_pollution_df['coord'].apply(split_coord)

# Drop the original 'coord' column
air_pollution_df.drop('coord',axis=1,inplace=True)

In [ ]:
cities_df = pd.DataFrame(CITIES)
cities_df.columns

Index(['city', 'lat', 'lon', 'population'], dtype='object')

In [ ]:
new_columns_name = ['city', 'latitude', 'longitude', 'population']
cities_df.columns= new_columns_name
cities_df

,city,latitude,longitude,population
0,Tokyo,35.6895,139.6917,13929286
1,Delhi,28.7041,77.1025,27882721
2,Shanghai,31.2304,121.4737,24256800
3,São Paulo,-23.5505,-46.6333,21571281
4,Mumbai,19.0760,72.8777,21558265
5,Mexico City,19.4326,-99.1332,21782378
6,Beijing,39.9042,116.4074,21516000
7,Osaka,34.6937,135.5023,19222665
8,New York,40.7128,-74.0060,18804000
9,Cairo,30.0444,31.2357,20095952


In [ ]:
air_pollution_df = pd.merge(air_pollution_df,cities_df, left_on='latitude',right_on='latitude')


In [ ]:
#air_pollution_df.drop('longitude_y',axis=1,inplace=True)
air_pollution_df

,dt,main.aqi,components.co,components.no,components.no2,components.o3,components.so2,components.pm2_5,components.pm10,components.nh3,longitude_x,latitude,city,longitude_y,population
0,1641016800,2,270.37,1.98,12.68,87.26,12.76,0.62,1.38,0.57,139.6917,35.6895,Tokyo,139.6917,13929286
1,1641020400,1,280.38,1.31,20.05,78.68,16.93,0.78,1.79,0.77,139.6917,35.6895,Tokyo,139.6917,13929286
2,1641024000,1,297.07,0.12,27.42,69.38,20.03,0.94,2.09,1.01,139.6917,35.6895,Tokyo,139.6917,13929286
3,1641027600,1,310.42,0.01,30.85,62.94,21.93,1.06,2.24,1.19,139.6917,35.6895,Tokyo,139.6917,13929286
4,1641031200,1,320.43,0.02,36.67,55.79,26.23,1.17,2.48,1.38,139.6917,35.6895,Tokyo,139.6917,13929286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85915,1672534800,2,317.10,0.02,23.99,28.61,12.76,23.35,48.37,2.50,31.2357,30.0444,Cairo,31.2357,20095952
85916,1672538400,2,313.76,0.02,22.28,27.18,12.28,23.23,43.94,2.47,31.2357,30.0444,Cairo,31.2357,20095952
85917,1672542000,2,310.42,0.03,21.25,28.25,12.16,22.07,39.83,2.41,31.2357,30.0444,Cairo,31.2357,20095952
85918,1672545600,2,313.76,0.04,22.62,28.25,13.83,20.83,36.78,2.53,31.2357,30.0444,Cairo,31.2357,20095952


In [ ]:
air_pollution_df['dt'] = pd.to_datetime(air_pollution_df['dt'],unit='s',origin='unix')
air_pollution_df['dt'] = air_pollution_df['dt'].dt.strftime('%Y-%m-%dT%H:%M')

In [ ]:
air_pollution_df.drop('longitude_y',axis=1,inplace=True)


In [ ]:
updated_pollution_df = air_pollution_df


In [ ]:
updated_pollution_df = updated_pollution_df.rename(columns={'dt':'date', 'city':'city', 'longitude_x':'longitude', 'latitude':'latitude', 'main.aqi':'AQI', 'components.co':'CO', 'components.no':'NO', 'components.no2':'NO2',
       'components.o3':'O3', 'components.so2':'SO2', 'components.pm2_5':'PM2_5', 'components.pm10':'PM10', 'components.nh3':'NH3', 'population':'population'})

In [ ]:
updated_pollution_df = updated_pollution_df[['date','longitude', 'latitude', 'city', 'population', 'AQI', 'CO', 'NO', 'NO2', 'O3', 'SO2', 'PM2_5', 'PM10', 'NH3',
       ]]
updated_pollution_df.sort_values(by='date')


,date,longitude,latitude,city,population,AQI,CO,NO,NO2,O3,SO2,PM2_5,PM10,NH3
0,2022-01-01T06:00,139.6917,35.6895,Tokyo,13929286,2,270.37,1.98,12.68,87.26,12.76,0.62,1.38,0.57
8592,2022-01-01T06:00,77.1025,28.7041,Delhi,27882721,5,2243.04,13.41,37.01,55.07,19.79,297.30,362.12,10.39
60144,2022-01-01T06:00,135.5023,34.6937,Osaka,19222665,1,307.08,8.05,26.39,64.37,20.27,2.50,5.01,0.86
77328,2022-01-01T06:00,31.2357,30.0444,Cairo,20095952,4,707.63,59.01,37.36,0.18,45.78,45.60,62.10,3.80
68736,2022-01-01T06:00,-74.0060,40.7128,New York,18804000,5,1161.58,112.65,45.24,0.00,9.18,51.33,59.05,3.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25775,2023-01-01T05:00,121.4737,31.2304,Shanghai,24256800,5,13137.82,786.78,216.60,0.50,2197.27,551.18,654.80,4.37
17183,2023-01-01T05:00,77.1025,28.7041,Delhi,27882721,5,2590.18,25.93,33.93,21.46,22.17,231.12,295.90,9.12
8591,2023-01-01T05:00,139.6917,35.6895,Tokyo,13929286,3,467.30,96.56,97.33,17.52,122.07,34.87,42.62,1.50
77327,2023-01-01T05:00,-74.0060,40.7128,New York,18804000,2,427.25,8.61,43.18,0.21,5.13,9.20,10.58,0.68


In [ ]:
updated_pollution_df.to_csv('air_pollution.csv')

In [ ]:
import glob


In [ ]:
# path where the CSV files are located
path = r'City_csv'

In [ ]:
# get a list of all CSV files in the directory
all_files = glob.glob(path + "/*.csv")

In [ ]:
# combine all CSV files into one dataframe
df = pd.concat((pd.read_csv(f) for f in all_files))

In [ ]:
# write the combined data to a new CSV file
df.to_csv("combined_data.csv", index=False)

In [ ]:
weather_data = pd.read_csv("combined_data.csv")

In [ ]:
weather_data_df = pd.DataFrame(weather_data)

In [ ]:
weather_data_df.rename(columns={'time':'date'}, inplace=True)

In [ ]:
weather_data_df

,date,temperature_2m (°C),relativehumidity_2m (%),precipitation (mm),rain (mm),cloudcover (%),cloudcover_low (%),cloudcover_mid (%),cloudcover_high (%),windspeed_10m (km/h),windspeed_100m (km/h),winddirection_10m (°),winddirection_100m (°),windgusts_10m (km/h),latitude,longitude,elevation,utc_offset_seconds,timezone,timezone_abbreviation
0,2022-01-01T06:00,6.6,25.0,0.0,0.0,0,0,0,0,7.1,9.5,210.0,205.0,34.2,39.9042,116.4074,47,0,GMT,GMT
1,2022-01-01T07:00,6.2,21.0,0.0,0.0,0,0,0,0,7.0,9.4,235.0,227.0,38.2,39.9042,116.4074,47,0,GMT,GMT
2,2022-01-01T08:00,4.5,23.0,0.0,0.0,0,0,0,0,6.9,13.3,276.0,267.0,35.3,39.9042,116.4074,47,0,GMT,GMT
3,2022-01-01T09:00,2.0,25.0,0.0,0.0,0,0,0,0,9.3,21.3,283.0,274.0,30.2,39.9042,116.4074,47,0,GMT,GMT
4,2022-01-01T10:00,0.1,26.0,0.0,0.0,0,0,0,0,10.1,22.7,274.0,267.0,29.2,39.9042,116.4074,47,0,GMT,GMT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87775,2023-01-01T19:00,1.3,87.0,0.0,0.0,23,3,34,0,3.4,10.3,342.0,12.0,6.8,35.6895,139.6917,43,0,GMT,GMT
87776,2023-01-01T20:00,1.3,85.0,0.0,0.0,21,5,28,0,4.0,10.8,360.0,15.0,6.1,35.6895,139.6917,43,0,GMT,GMT
87777,2023-01-01T21:00,0.5,86.0,0.0,0.0,29,10,33,0,5.2,12.5,16.0,18.0,7.6,35.6895,139.6917,43,0,GMT,GMT
87778,2023-01-01T22:00,0.2,86.0,0.0,0.0,22,7,26,0,6.4,4.0,223.0,333.0,8.3,35.6895,139.6917,43,0,GMT,GMT


In [ ]:
updated_pollution_df

,date,longitude,latitude,city,population,AQI,CO,NO,NO2,O3,SO2,PM2_5,PM10,NH3
0,2022-01-01T06:00,139.6917,35.6895,Tokyo,13929286,2,270.37,1.98,12.68,87.26,12.76,0.62,1.38,0.57
1,2022-01-01T07:00,139.6917,35.6895,Tokyo,13929286,1,280.38,1.31,20.05,78.68,16.93,0.78,1.79,0.77
2,2022-01-01T08:00,139.6917,35.6895,Tokyo,13929286,1,297.07,0.12,27.42,69.38,20.03,0.94,2.09,1.01
3,2022-01-01T09:00,139.6917,35.6895,Tokyo,13929286,1,310.42,0.01,30.85,62.94,21.93,1.06,2.24,1.19
4,2022-01-01T10:00,139.6917,35.6895,Tokyo,13929286,1,320.43,0.02,36.67,55.79,26.23,1.17,2.48,1.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85915,2023-01-01T01:00,31.2357,30.0444,Cairo,20095952,2,317.10,0.02,23.99,28.61,12.76,23.35,48.37,2.50
85916,2023-01-01T02:00,31.2357,30.0444,Cairo,20095952,2,313.76,0.02,22.28,27.18,12.28,23.23,43.94,2.47
85917,2023-01-01T03:00,31.2357,30.0444,Cairo,20095952,2,310.42,0.03,21.25,28.25,12.16,22.07,39.83,2.41
85918,2023-01-01T04:00,31.2357,30.0444,Cairo,20095952,2,313.76,0.04,22.62,28.25,13.83,20.83,36.78,2.53


In [ ]:
data_merged = pd.merge(updated_pollution_df,weather_data_df, on=['date','longitude','latitude'], how="left")

In [ ]:
data_merged.drop('timezone_abbreviation',axis=1,inplace=True)
data_merged.drop('timezone',axis=1,inplace=True)
data_merged.drop('utc_offset_seconds',axis=1,inplace=True)

In [ ]:
data_merged

,date,longitude,latitude,city,population,AQI,CO,NO,NO2,O3,...,cloudcover (%),cloudcover_low (%),cloudcover_mid (%),cloudcover_high (%),windspeed_10m (km/h),windspeed_100m (km/h),winddirection_10m (°),winddirection_100m (°),windgusts_10m (km/h),elevation
0,2022-01-01T06:00,139.6917,35.6895,Tokyo,13929286,2,270.37,1.98,12.68,87.26,...,0,0,0,0,16.8,22.0,211.0,212.0,43.6,43
1,2022-01-01T07:00,139.6917,35.6895,Tokyo,13929286,1,280.38,1.31,20.05,78.68,...,0,0,0,0,16.7,25.1,187.0,187.0,35.3,43
2,2022-01-01T08:00,139.6917,35.6895,Tokyo,13929286,1,297.07,0.12,27.42,69.38,...,0,0,0,0,15.9,25.7,177.0,175.0,34.9,43
3,2022-01-01T09:00,139.6917,35.6895,Tokyo,13929286,1,310.42,0.01,30.85,62.94,...,0,0,0,0,15.2,24.9,185.0,177.0,31.7,43
4,2022-01-01T10:00,139.6917,35.6895,Tokyo,13929286,1,320.43,0.02,36.67,55.79,...,0,0,0,0,14.4,25.9,184.0,170.0,29.5,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85915,2023-01-01T01:00,31.2357,30.0444,Cairo,20095952,2,317.10,0.02,23.99,28.61,...,17,2,0,51,9.7,21.9,NaN,NaN,NaN,22
85916,2023-01-01T02:00,31.2357,30.0444,Cairo,20095952,2,313.76,0.02,22.28,27.18,...,14,1,0,43,10.1,22.5,NaN,NaN,NaN,22
85917,2023-01-01T03:00,31.2357,30.0444,Cairo,20095952,2,310.42,0.03,21.25,28.25,...,19,8,1,36,9.7,22.5,NaN,NaN,NaN,22
85918,2023-01-01T04:00,31.2357,30.0444,Cairo,20095952,2,313.76,0.04,22.62,28.25,...,19,8,0,39,9.4,22.6,NaN,NaN,NaN,22


In [ ]:
final_set = data_merged.dropna()

In [ ]:
final_set

,date,longitude,latitude,city,population,AQI,CO,NO,NO2,O3,...,cloudcover (%),cloudcover_low (%),cloudcover_mid (%),cloudcover_high (%),windspeed_10m (km/h),windspeed_100m (km/h),winddirection_10m (°),winddirection_100m (°),windgusts_10m (km/h),elevation
0,2022-01-01T06:00,139.6917,35.6895,Tokyo,13929286,2,270.37,1.98,12.68,87.26,...,0,0,0,0,16.8,22.0,211.0,212.0,43.6,43
1,2022-01-01T07:00,139.6917,35.6895,Tokyo,13929286,1,280.38,1.31,20.05,78.68,...,0,0,0,0,16.7,25.1,187.0,187.0,35.3,43
2,2022-01-01T08:00,139.6917,35.6895,Tokyo,13929286,1,297.07,0.12,27.42,69.38,...,0,0,0,0,15.9,25.7,177.0,175.0,34.9,43
3,2022-01-01T09:00,139.6917,35.6895,Tokyo,13929286,1,310.42,0.01,30.85,62.94,...,0,0,0,0,15.2,24.9,185.0,177.0,31.7,43
4,2022-01-01T10:00,139.6917,35.6895,Tokyo,13929286,1,320.43,0.02,36.67,55.79,...,0,0,0,0,14.4,25.9,184.0,170.0,29.5,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68731,2023-01-01T01:00,135.5023,34.6937,Osaka,19222665,2,347.14,10.06,30.85,45.06,...,17,17,3,0,12.5,17.0,252.0,253.0,30.6,14
68732,2023-01-01T02:00,135.5023,34.6937,Osaka,19222665,2,320.43,9.95,25.70,53.64,...,22,23,2,0,15.3,20.8,261.0,261.0,35.3,14
68733,2023-01-01T03:00,135.5023,34.6937,Osaka,19222665,2,307.08,9.05,22.96,59.37,...,39,37,9,0,17.0,22.8,265.0,265.0,38.5,14
68734,2023-01-01T04:00,135.5023,34.6937,Osaka,19222665,2,297.07,7.04,20.39,62.94,...,90,73,40,0,18.7,25.6,271.0,272.0,40.0,14


In [ ]:
# Convert the 'date' column to a datetime format
final_set['date'] = pd.to_datetime(final_set['date'])

# Set the 'date' column as the index
final_set = final_set.set_index('date')

daily_mean = final_set.groupby('city').resample('D').mean()


KeyError: 'date'

In [ ]:
daily_mean.dropna()

longitude  latitude  population       AQI           CO  \
city    date                                                                 
Beijing 2022-01-01   116.4074   39.9042  21516000.0  4.111111   769.562778   
        2022-01-02   116.4074   39.9042  21516000.0  5.000000  3797.371250   
        2022-01-03   116.4074   39.9042  21516000.0  5.000000  3721.157083   
        2022-01-04   116.4074   39.9042  21516000.0  5.000000  4896.641250   
        2022-01-05   116.4074   39.9042  21516000.0  5.000000  5174.796250   
...                       ...       ...         ...       ...          ...   
Tokyo   2022-12-28   139.6917   35.6895  13929286.0  3.125000   771.880000   
        2022-12-29   139.6917   35.6895  13929286.0  2.625000   544.350417   
        2022-12-30   139.6917   35.6895  13929286.0  2.541667   541.567500   
        2022-12-31   139.6917   35.6895  13929286.0  2.583333   588.297083   
        2023-01-01   139.6917   35.6895  13929286.0  2.666667   550.746667   

                            NO         NO2         O3         SO2       PM2_5  \
city    date                                                                    
Beijing 2022-01-01   10.766111   50.610556  26.118889   44.980556   47.486111   
        2022-01-02  185.613333  101.475833   4.031667  231.822083  290.419583   
        2022-01-03  221.449583  107.501250   4.019583  195.861250  298.490833   
        2022-01-04  301.599583  110.500833   1.786667  232.220417  399.904167   
        2022-01-05  334.345417  133.663750   2.645417  298.659167  444.687083   
...                        ...         ...        ...         ...         ...   
Tokyo   2022-12-28  214.539167   98.049167   2.054583  115.552917   34.445833   
        2022-12-29   94.520417   79.713333   3.005000   83.982500   14.660417   
        2022-12-30  104.864583   77.199167   3.588333   85.681250   10.602500   
        2022-12-31  122.003333   77.198750   2.209167  101.209167   15.521667   
        2023-01-01  119.060000   76.885000  12.218333   95.683333   26.610000   

                    ...  cloudcover (%)  cloudcover_low (%)  \
city    date        ...                                       
Beijing 2022-01-01  ...        0.000000            0.000000   
        2022-01-02  ...       19.583333            0.000000   
        2022-01-03  ...        0.000000            0.000000   
        2022-01-04  ...        1.583333            0.291667   
        2022-01-05  ...        2.625000            2.833333   
...                 ...             ...                 ...   
Tokyo   2022-12-28  ...       10.250000            7.791667   
        2022-12-29  ...       13.625000            3.875000   
        2022-12-30  ...       16.708333           10.875000   
        2022-12-31  ...       20.750000           16.875000   
        2023-01-01  ...        4.666667            5.000000   

                    cloudcover_mid (%)  cloudcover_high (%)  \
city    date                                                  
Beijing 2022-01-01            0.000000             0.000000   
        2022-01-02           14.541667            36.083333   
        2022-01-03            0.000000             0.000000   
        2022-01-04            0.000000             4.375000   
        2022-01-05            0.000000             0.416667   
...                                ...                  ...   
Tokyo   2022-12-28            4.458333             1.541667   
        2022-12-29           16.666667             0.000000   
        2022-12-30           11.458333             0.000000   
        2022-12-31            9.166667             0.000000   
        2023-01-01            0.166667             0.000000   

                    windspeed_10m (km/h)  windspeed_100m (km/h)  \
city    date                                                      
Beijing 2022-01-01              6.577778              17.827778   
        2022-01-02              6.508333               9.237500   
        2022-01-03              6.137500   

In [ ]:
daily_mean.to_csv('daily_mean.csv')